# Projeto de Predição de Sucesso de Startups

## 1. Introdução

Este notebook documenta o processo de desenvolvimento de um modelo preditivo para identificar o sucesso ou insucesso de startups, com base em um conjunto de dados fornecido. O objetivo é seguir as etapas de um projeto de Machine Learning, desde a exploração dos dados até a avaliação e otimização do modelo, atendendo aos critérios de avaliação estabelecidos.

## 2. Carregamento e Exploração Inicial dos Dados

Primeira etapa: carregar os dados de treino e teste para compreender sua estrutura e características.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# Carregar os dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print('Dados de Treino:')
print(train.head())
print('\nInformações do Treino:')
print(train.info())
print('\nDados de Teste:')
print(test.head())
print('\nInformações do Teste:')
print(test.info())

## 3. Limpeza e Tratamento de Valores Nulos

A limpeza e o tratamento de valores nulos e *outliers* são etapas cruciais para garantir a qualidade dos dados e a robustez do modelo. O código base (`05.py` corrigido) implementa uma estratégia avançada para lidar com esses aspectos.

### 3.1. Tratamento de Valores Ausentes (NaN)

A função `advanced_preprocessing_pipeline` trata os valores ausentes de forma inteligente:

* **Features de Localização e Indústria**: Para features categóricas como `state_code`, `city`, `industry`, etc., os valores ausentes são preenchidos com a **moda** (valor mais frequente) da coluna.

* **Features Financeiras**: Para `founded_year`, `total_funding_usd`, `employee_count`, e outras features financeiras críticas, é utilizada a **mediana** para imputação, que é mais robusta contra *outliers*.

* **Outras Features Numéricas**: Para as demais features numéricas, a imputação é feita utilizando a **mediana** (`SimpleImputer(strategy='median')`). A mediana é preferível à média em distribuições assimétricas, comuns em dados de startups.

In [ ]:
# Verificar valores ausentes
print('Valores ausentes no conjunto de treino:')
missing_train = train.isnull().sum()
print(missing_train[missing_train > 0].sort_values(ascending=False))

print('\nValores ausentes no conjunto de teste:')
missing_test = test.isnull().sum()
print(missing_test[missing_test > 0].sort_values(ascending=False))

# Análise básica da variável target
print('\nDistribuição da variável target:')
print(train['labels'].value_counts())
print('\nProporção:')
print(train['labels'].value_counts(normalize=True))

## 4. Análise Exploratória e Visualizações

Análise das principais características dos dados para entender melhor os padrões que podem indicar o sucesso de startups.

In [ ]:
# Configurar o estilo dos gráficos
plt.style.use('default')
sns.set_palette('husl')

# Análise da distribuição da variável target
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribuição da variável target
train['labels'].value_counts().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Distribuição da Variável Target')
axes[0,0].set_xlabel('Labels (0=Insucesso, 1=Sucesso)')

# Distribuição por indústria
if 'industry' in train.columns:
    top_industries = train['industry'].value_counts().head(10)
    top_industries.plot(kind='barh', ax=axes[0,1])
    axes[0,1].set_title('Top 10 Indústrias')

# Distribuição por estado
if 'state_code' in train.columns:
    top_states = train['state_code'].value_counts().head(10)
    top_states.plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Top 10 Estados')

# Distribuição de funding
if 'total_funding_usd' in train.columns:
    train['total_funding_usd'].hist(bins=50, ax=axes[1,1], alpha=0.7)
    axes[1,1].set_title('Distribuição do Funding Total (USD)')
    axes[1,1].set_xlabel('Total Funding USD')

plt.tight_layout()
plt.show()

## 5. Pré-processamento e Preparação dos Dados

Implementação do pipeline de pré-processamento baseado no código `05.py` otimizado.

In [ ]:
# Função de pré-processamento baseada no 05.py
def preprocess_data(train_df, test_df):
    # Fazer cópias para não modificar os originais
    train_proc = train_df.copy()
    test_proc = test_df.copy()
    
    # Separar features e target
    if 'labels' in train_proc.columns:
        y = train_proc['labels']
        train_proc = train_proc.drop(['labels'], axis=1)
    
    # Remover colunas de ID se existirem
    id_cols = ['id', 'Unnamed: 0']
    for col in id_cols:
        if col in train_proc.columns:
            train_proc = train_proc.drop(col, axis=1)
        if col in test_proc.columns:
            test_proc = test_proc.drop(col, axis=1)
    
    # Identificar colunas categóricas e numéricas
    categorical_cols = train_proc.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = train_proc.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Tratar valores ausentes
    # Para colunas categóricas: preencher com moda
    for col in categorical_cols:
        mode_val = train_proc[col].mode()[0] if len(train_proc[col].mode()) > 0 else 'Unknown'
        train_proc[col].fillna(mode_val, inplace=True)
        test_proc[col].fillna(mode_val, inplace=True)
    
    # Para colunas numéricas: preencher com mediana
    for col in numerical_cols:
        median_val = train_proc[col].median()
        train_proc[col].fillna(median_val, inplace=True)
        test_proc[col].fillna(median_val, inplace=True)
    
    # Encoding de variáveis categóricas
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        # Fit no conjunto combinado para garantir consistência
        combined_values = pd.concat([train_proc[col], test_proc[col]]).unique()
        le.fit(combined_values)
        
        train_proc[col] = le.transform(train_proc[col])
        test_proc[col] = le.transform(test_proc[col])
        label_encoders[col] = le
    
    return train_proc, test_proc, y, label_encoders

# Aplicar pré-processamento
X_train_proc, X_test_proc, y_train, encoders = preprocess_data(train, test)

print(f'Shape do treino processado: {X_train_proc.shape}')
print(f'Shape do teste processado: {X_test_proc.shape}')
print(f'Colunas após processamento: {len(X_train_proc.columns)}')

## 6. Treinamento e Avaliação do Modelo

Implementação do modelo ensemble baseado no código otimizado.

In [ ]:
# Dividir dados de treino para validação
X_train, X_val, y_train_split, y_val = train_test_split(
    X_train_proc, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test_proc)

# Criar modelo ensemble
rf_model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    class_weight='balanced',
    random_state=42
)

lr_model = LogisticRegression(
    class_weight='balanced', 
    random_state=42, 
    max_iter=1000
)

svm_model = SVC(
    class_weight='balanced', 
    probability=True, 
    random_state=42
)

# Ensemble com voting
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf_model),
        ('lr', lr_model),
        ('svm', svm_model)
    ],
    voting='soft'
)

# Treinar o modelo
print('Treinando modelo ensemble...')
ensemble.fit(X_train_scaled, y_train_split)

# Avaliar no conjunto de validação
val_predictions = ensemble.predict(X_val_scaled)
val_probabilities = ensemble.predict_proba(X_val_scaled)[:, 1]

# Métricas de avaliação
print('\nResultados na Validação:')
print(classification_report(y_val, val_predictions))
print(f'\nROC AUC Score: {roc_auc_score(y_val, val_probabilities):.4f}')

# Matriz de confusão
cm = confusion_matrix(y_val, val_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão - Validação')
plt.ylabel('Valor Real')
plt.xlabel('Valor Predito')
plt.show()

## 7. Predições Finais e Submissão

Geração das predições finais para o conjunto de teste.

In [ ]:
# Treinar modelo final com todos os dados de treino
print('Treinando modelo final com todos os dados...')
final_scaler = StandardScaler()
X_train_final = final_scaler.fit_transform(X_train_proc)
X_test_final = final_scaler.transform(X_test_proc)

# Treinar modelo final
final_ensemble = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=42)),
        ('lr', LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)),
        ('svm', SVC(class_weight='balanced', probability=True, random_state=42))
    ],
    voting='soft'
)

final_ensemble.fit(X_train_final, y_train)

# Fazer predições finais
final_predictions = final_ensemble.predict(X_test_final)
final_probabilities = final_ensemble.predict_proba(X_test_final)[:, 1]

# Criar arquivo de submissão
submission = pd.DataFrame({
    'id': test['id'],
    'labels': final_predictions
})

submission.to_csv('submission.csv', index=False)
print('Arquivo de submissão gerado: submission.csv')

# Mostrar estatísticas da predição
print(f'\nDistribuição das predições:')
print(submission['labels'].value_counts())
print(f'\nProporção de sucessos preditos: {submission["labels"].mean():.3f}')